# 01_exploration.ipynb - Data Exploration

In [ ]:
import os
import pandas as pd
import glob

# Define base paths
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Raw data folder
DATA_RAW = os.path.join(BASE_DIR, "data", "raw", "pickup_data")

#  Figures folder
FIG_DIR = os.path.abspath(os.path.join(os.getcwd(), "figures"))
os.makedirs(FIG_DIR, exist_ok=True)

print(f"📁 Figures will be saved to: {FIG_DIR}")

# Load and combine all monthly CSV file
all_files = glob.glob(os.path.join(DATA_RAW, "*.csv"))
print(f"🔍 Found {len(all_files)} CSV file(s) in {DATA_RAW}")
print(all_files)

if all_files:
    df = pd.concat([pd.read_csv(f) for f in all_files], ignore_index=True)
    print(f"✅ Combined dataset shape: {df.shape}")
    display(df.head(10))
    df.info()
else:
    print("⚠️ No CSV files found. Please check the data/raw/pickup_data folder path.")

## I.  Data Cleaning

In [ ]:
# Count true duplicates:Date/Time, Lat, Lon, and Base are exactly the same
num_duplicates = df.duplicated(subset=['Date/Time','Lat','Lon','Base']).sum()
print(f"Number of duplicates based on original columns: {num_duplicates}")

# Check duplicates
duplicate_rows = df[df.duplicated(subset=['Date/Time','Lat','Lon','Base'], keep=False)]
duplicate_rows.head(10)

In [ ]:
# Remove duplicates 
df = df.drop_duplicates(subset=['Date/Time','Lat','Lon','Base'])
num_duplicates_after = df.duplicated(subset=['Date/Time','Lat','Lon','Base']).sum()
print(f"Remaining duplicates: {num_duplicates_after}")

# Check for missing values in each column
df.isnull().sum()

# Quick check
print(df.info())
print(df.describe())

## II. Data Exploration

#### Initial feature extraction

In [ ]:
# Convert Date/Time
df['Date/Time'] = pd.to_datetime(df['Date/Time'])
df['Hour'] = df['Date/Time'].dt.hour
df['Day'] = df['Date/Time'].dt.day
df['Weekday'] = df['Date/Time'].dt.weekday  # Monday=0, Sunday=6
df['Month'] = df['Date/Time'].dt.month  # 1 = Jan, 12 = Dec

### A. Temporal Data

In [ ]:
# Get time range
start_date = df['Date/Time'].min()
end_date = df['Date/Time'].max()
print(f"🗓️ Date range: {start_date} → {end_date}")

# Check number of days covered
days_covered = (end_date - start_date).days
print(f"Total days covered: {days_covered}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import calendar

# Pickups per hour
plt.figure(figsize=(10,5))
sns.countplot(x='Hour', data=df, color='skyblue')
plt.title('Uber Pickups by Hour of the Day')
plt.xlabel('Hour of the Day (0–23)')
plt.ylabel('Number of Pickups')
plt.xticks(range(0,24))
plt.tight_layout()
plt.show()


# Pickups per weekday
weekday_labels = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']

plt.figure(figsize=(10,5))
sns.countplot(x='Weekday', data=df, color='lightgreen')
plt.title('Uber Pickups by Weekday')
plt.xlabel('Weekday')
plt.ylabel('Number of Pickups')
plt.xticks(ticks=range(7), labels=weekday_labels)
plt.tight_layout()
plt.show()


# Average pickups per calendar day (across months)
calendar_day = df.groupby('Day').size().reset_index(name='Total_Pickups')
calendar_day['Avg_Pickups'] = calendar_day['Total_Pickups'] / df['Month'].nunique()

plt.figure(figsize=(12,5))
sns.barplot(x='Day', y='Avg_Pickups', data=calendar_day, color='skyblue')
plt.title("Average Pickups per Calendar Day (Across Months)")
plt.xlabel("Day of Month")
plt.ylabel("Average Number of Pickups")
plt.tight_layout()
plt.show()


# Daily total pickups over time
calendar_day_full = df.groupby(df['Date/Time'].dt.date).size().reset_index(name='Pickups')

plt.figure(figsize=(14,5))
sns.lineplot(x='Date/Time', y='Pickups', data=calendar_day_full, color='steelblue')
plt.title("Daily Total Pickups Over Time")
plt.xlabel("Date")
plt.ylabel("Number of Pickups")
plt.tight_layout()
plt.show()

# Heatmap: Hour vs Weekday
pivot = df.pivot_table(index='Hour', columns='Weekday', values='Date/Time', aggfunc='count', fill_value=0)

# Normalize within each weekday (column) to show % of daily pickups
pivot_norm = pivot.div(pivot.sum(axis=0), axis=1) * 100  # convert to percentages

# Rename columns to weekday names
weekday_labels = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
pivot_norm.columns = weekday_labels

# Plot heatmap
plt.figure(figsize=(10,6))
sns.heatmap(
    pivot_norm,
    cmap="YlOrRd",
    annot=True,
    fmt=".1f",
    linewidths=0.3,
    cbar_kws={'label': 'Percentage of Daily Pickups (%)'}
)
plt.title("Normalized Heatmap of Uber Pickups: Hour vs Weekday", fontsize=14, pad=12)
plt.xlabel("Weekday", fontsize=12)
plt.ylabel("Hour of Day", fontsize=12)
plt.yticks(rotation=0)  # keep hour labels horizontal
plt.tight_layout()
plt.show()

# Most frequent pickup patterns
most_common_hour = df['Hour'].mode()[0]
pickup_count_hour = df['Hour'].value_counts().head(1)
print("Most frequent pickup hour:")
print(pickup_count_hour, '\n')

most_common_day = df['Day'].mode()[0]
pickup_count_day = df['Day'].value_counts().head(1)
print("Most frequent pickup day (calendar day of month):")
print(pickup_count_day, '\n')

most_common_weekday = df['Weekday'].mode()[0]
pickup_count_weekday = df['Weekday'].value_counts().head(1)
print("Most frequent weekday (0=Monday, 6=Sunday):")
print(pickup_count_weekday)
print(f"That’s a {calendar.day_name[most_common_weekday]}.\n")

# Most frequent (hour, day) combination
most_common_hour_day = df.groupby(['Day', 'Hour']).size().reset_index(name='Count')
top_hour_day = most_common_hour_day.sort_values(by='Count', ascending=False).head(1)
print("Most frequent hour–day combination:")
print(top_hour_day)

Hourly Pattern: A clear diurnal rhythm is visible, with minimal activity during early morning hours (2–5 AM) and distinct peaks during the evening commute (16:00–19:00). This pattern reflects strong commuter-driven demand and emphasizes the city’s daily mobility cycles.

Weekly Pattern: Demand remains relatively stable during the weekday, with a noticeable rise on Thursday and Friday evenings as social and leisure activities increase. Weekends show later but sustained evening peaks, indicating different behavioral patterns compared to weekdays.

Long-Term Trend: Over time, a steady upward trend is observed in overall demand, suggesting either service expansion, urban growth, change of profile of customers, or seasonal improvements (e.g., tourism or weather-related effects). This progression underscores the evolving nature of urban mobility.

Temporal predictors such as hour of day, weekday, and overall trend show the strongest explanatory power for short-term demand forecasting. In contrast, day-of-month adds little predictive value and can be safely deprioritized in modeling pipelines.

### B. Spatial Data

#### Outliers

In [ ]:
import pandas as pd

# Explore coordinate range
lat_min, lat_max = df['Lat'].min(), df['Lat'].max()
lon_min, lon_max = df['Lon'].min(), df['Lon'].max()

print(f"📍 Latitude range:  {lat_min:.4f} → {lat_max:.4f}")
print(f"📍 Longitude range: {lon_min:.4f} → {lon_max:.4f}")

# Geographical center of all pickup points
print(f"📌 Average latitude:  {df['Lat'].mean():.4f}")
print(f"📌 Average longitude: {df['Lon'].mean():.4f}")

# Define expected NYC bounds (rough operational area)
lat_bounds = (40.5, 41.0)
lon_bounds = (-74.3, -73.7)

# - Find points outside NYC bounding box
outliers_bounds = df[
    (df['Lat'] < lat_bounds[0]) | (df['Lat'] > lat_bounds[1]) |
    (df['Lon'] < lon_bounds[0]) | (df['Lon'] > lon_bounds[1])
]
print(f"🚧 Outliers outside NYC bounds: {len(outliers_bounds)}")

# Detect outliers more robustly using IQR (Interquartile Range)
Q1_lat, Q3_lat = df['Lat'].quantile([0.25, 0.75])
Q1_lon, Q3_lon = df['Lon'].quantile([0.25, 0.75])

IQR_lat = Q3_lat - Q1_lat
IQR_lon = Q3_lon - Q1_lon

lat_min_iqr, lat_max_iqr = Q1_lat - 1.5 * IQR_lat, Q3_lat + 1.5 * IQR_lat
lon_min_iqr, lon_max_iqr = Q1_lon - 1.5 * IQR_lon, Q3_lon + 1.5 * IQR_lon

# Filter outliers based on IQR
outliers_iqr = df[
    (df['Lat'] < lat_min_iqr) | (df['Lat'] > lat_max_iqr) |
    (df['Lon'] < lon_min_iqr) | (df['Lon'] > lon_max_iqr)
]

print(f"📉 Outliers detected via IQR: {len(outliers_iqr)}")

# -Filtered dataset (within NYC bounds and IQR range)
nyc_df = df[
    (df['Lat'].between(lat_min_iqr, lat_max_iqr)) &
    (df['Lon'].between(lon_min_iqr, lon_max_iqr))
]

print(f"✅ Remaining NYC data points: {len(nyc_df)} ({len(nyc_df)/len(df)*100:.2f}% of total)")


The dataset covers pickup coordinates confirm that nearly all trips fall within the New York City metropolitan area. The average pickup location (40.739°, −73.973°) aligns with central Manhattan, highlighting strong urban concentration.

A small share of trips (~0.6%) occurred outside city bounds, while the IQR-based filter flagged an additional ~11% of statistically extreme points likely caused by long-distance rides beyond operational relevance.

After cleaning, 3.94 million valid pickups (≈88.6%) remain, providing a robust and geographically consistent dataset for subsequent clustering and demand forecasting.

#### Top 10 Busiest Pickup Locations (All Days, All Hours)

In [ ]:
from geopy.geocoders import Nominatim
import folium
import time
import pandas as pd

nyc_df['Lat_round'] = nyc_df['Lat'].round(3)
nyc_df['Lon_round'] = nyc_df['Lon'].round(3)

# Find the Top 10 busiest pickup points
top_locations = (
    nyc_df.groupby(['Lat_round', 'Lon_round'])
          .size()
          .reset_index(name='Count')
          .sort_values(by='Count', ascending=False)
          .head(10)
          .reset_index(drop=True)
)

# Reverse Geocode the coordinates
geolocator = Nominatim(user_agent="nyc_taxi_analysis")
top_locations['Address'] = None  # prepare empty column

for i, row in top_locations.iterrows():
    try:
        location = geolocator.reverse((row['Lat_round'], row['Lon_round']), exactly_one=True, timeout=10)
        top_locations.at[i, 'Address'] = location.address if location else "Unknown"
        time.sleep(1)  # prevent API blocking by Nominatim
    except Exception as e:
        top_locations.at[i, 'Address'] = f"Error: {e}"


display(top_locations[['Lat_round', 'Lon_round', 'Count', 'Address']])

# Interactive Folium Map
nyc_center = [40.7128, -74.0060]
m = folium.Map(location=nyc_center, zoom_start=11, tiles='CartoDB positron')

# Add each top location as a circle marker
for _, row in top_locations.iterrows():
    popup_text = f"""
    <b>Pickups:</b> {row['Count']}<br>
    <b>Coordinates:</b> ({row['Lat_round']:.4f}, {row['Lon_round']:.4f})<br>
    <b>Address:</b> {row['Address']}
    """
    folium.CircleMarker(
        location=[row['Lat_round'], row['Lon_round']],
        radius=6 + (row['Count'] / top_locations['Count'].max()) * 10,  # scaled radius
        color='navy',
        fill=True,
        fill_color='lightblue',
        fill_opacity=0.7,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(m)

# Save map to the Figures folder
map_path = os.path.join(FIG_DIR, "01_top10_hotspots.html")
m.save(map_path)

print(f"💾 Map saved to: {map_path}")

m

Spatial exploration after removing outliers (e.g. airport pickups) revealed that the majority of high-demand locations are primarily around Manhattan’s commercial and entertainment zones such as Chelsea, Midtown, and SoHo.

#### Spatial Clustering on NYC Pickup Points

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans

# Sample data for faster computation
sample_size = 100_000  # adjust if still slow
df_sample = nyc_df.sample(n=sample_size, random_state=42)[['Lat', 'Lon']]

print(f"Running elbow method on {len(df_sample):,} points...")

# Compute inertia for different K values
inertia = []
K = range(3, 16)  # test cluster numbers between 3 and 15

for k in K:
    kmeans = MiniBatchKMeans(n_clusters=k, batch_size=10_000, random_state=42)
    kmeans.fit(df_sample)
    inertia.append(kmeans.inertia_)

# Plot the elbow curve
plt.figure(figsize=(8,5))
plt.plot(K, inertia, 'bo-', linewidth=2, markersize=6)
plt.xlabel('Number of Clusters (k)', fontsize=12)
plt.ylabel('Inertia (Within-Cluster Sum of Squares)', fontsize=12)
plt.title('Elbow Method to Determine Optimal k', fontsize=14, pad=10)
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

The elbow plot indicates a clear inflection point around k = 7–8, after which the rate of decrease in within-cluster variance slows. This suggests that 7–8 clusters would sufficiently represent the main spatial patterns in the data. Beyond this point, adding more clusters yields only marginal improvements while increasing model complexity.

#### Kmeans model for 10 clusters

In [ ]:
from sklearn.cluster import MiniBatchKMeans
import numpy as np
import pandas as pd
import folium

# KMeans Clustering (MiniBatch for speed)
coords = nyc_df[['Lat', 'Lon']]

kmeans = MiniBatchKMeans(n_clusters=10, batch_size=10_000, random_state=42)
kmeans.fit(coords)

# Assign clusters
nyc_df['Cluster'] = kmeans.predict(coords)

# Cluster centroids (from model)
centroids = pd.DataFrame(kmeans.cluster_centers_, columns=['Centroid_Lat', 'Centroid_Lon'])
centroids['Cluster'] = range(10)

# - Summary stats per cluster
cluster_summary = (
    nyc_df.groupby('Cluster')
    .agg(
        Pickups=('Cluster', 'size'),
        Avg_Lat=('Lat', 'mean'),
        Avg_Lon=('Lon', 'mean')
    )
    .reset_index()
)

# Merge centroid and summary info
kmeans_summary = pd.merge(centroids, cluster_summary, on='Cluster', how='left')

# Add percentage of total pickups
total_pickups = len(nyc_df)
kmeans_summary['Pct_of_Total'] = (kmeans_summary['Pickups'] / total_pickups * 100).round(2)

# Compute cluster radius (approximate geographic spread)
def cluster_radius(cluster_id):
    cluster_id = int(cluster_id)  # ensure integer type
    cluster_points = nyc_df.loc[nyc_df['Cluster'] == cluster_id, ['Lat', 'Lon']]
    centroid = np.array([
        kmeans_summary.loc[kmeans_summary['Cluster'] == cluster_id, 'Centroid_Lat'].values[0],
        kmeans_summary.loc[kmeans_summary['Cluster'] == cluster_id, 'Centroid_Lon'].values[0]
    ])
    distances = np.sqrt(((cluster_points - centroid) ** 2).sum(axis=1))
    return np.mean(distances)

kmeans_summary['Cluster'] = kmeans_summary['Cluster'].astype(int)
kmeans_summary['Radius'] = kmeans_summary['Cluster'].apply(cluster_radius).round(4)

kmeans_summary = kmeans_summary[['Cluster','Pickups','Pct_of_Total','Avg_Lat','Avg_Lon','Radius']]

display(kmeans_summary)

#  Visualize clusters on a Folium map ---
nyc_center = [40.7128, -74.0060]
m = folium.Map(location=nyc_center, zoom_start=11, tiles='CartoDB positron')

colors = [
    'red','blue','green','purple','orange',
    'darkred','lightblue','darkgreen','gray','pink'
]

for _, row in kmeans_summary.iterrows():
    cluster_id = int(row['Cluster'])  # make sure it's int
    popup_text = f"""
    <b>Cluster:</b> {cluster_id}<br>
    <b>Pickups:</b> {row['Pickups']:,}<br>
    <b>Share of Total:</b> {row['Pct_of_Total']}%<br>
    <b>Avg Coordinates:</b> ({row['Avg_Lat']:.4f}, {row['Avg_Lon']:.4f})<br>
    <b>Radius:</b> {row['Radius']:.4f}°
    """
    folium.CircleMarker(
        location=[row['Avg_Lat'], row['Avg_Lon']],
        radius=8 + (row['Pickups'] / kmeans_summary['Pickups'].max()) * 8,
        color=colors[cluster_id % len(colors)],
        fill=True,
        fill_color=colors[cluster_id % len(colors)],
        fill_opacity=0.6,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(m)

# Save map to the Figures folder
k_map_path = os.path.join(FIG_DIR, "01_kmeans_clusters_nyc.html")
m.save(k_map_path)

print(f"💾 Map saved to: {k_map_path}")

m

#### NYC Community Districs

While the results confirmed that demand is highly concentrated around central Manhattan, cluster boundaries proved unstable and uneven.
To improve interpretability and align with operational planning, the analysis transitioned to New York City community districts as consistent geographic units for further demand modeling.

The dataset included was taken from https://www.nyc.gov/content/planning/pages/resources/datasets/community-districts.

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import os

# Convert DataFrame to GeoDataFrame
geometry = [Point(xy) for xy in zip(nyc_df['Lon'], nyc_df['Lat'])]
gdf = gpd.GeoDataFrame(nyc_df, geometry=geometry, crs="EPSG:4326")
print(f"✓ Converted nyc_df to GeoDataFrame with {len(gdf):,} pickup points")

# Define dynamic path (from inside notebooks folder)
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_RAW = os.path.join(BASE_DIR, "data", "raw", "district_nyc_data")

# Try loading a valid geographic file automatically
try:
    print(f"\nLooking for Community Districts files in: {DATA_RAW}")

    if not os.path.exists(DATA_RAW):
        raise FileNotFoundError(f"Path not found: {DATA_RAW}")

    geo_files = [
        f for f in os.listdir(DATA_RAW)
        if any(f.lower().endswith(ext) for ext in ['.shp', '.geojson', '.json', '.gpkg'])
    ]

    if geo_files:
        file_path = os.path.join(DATA_RAW, geo_files[0])
        nyc_cds = gpd.read_file(file_path)
        print(f"✓ Loaded geographic file: {geo_files[0]}")
    else:
        raise FileNotFoundError("No shapefile or GeoJSON found in data/raw/district_nyc_data")

    print(f"Loaded {len(nyc_cds)} community district polygons")
    print("Available columns:", list(nyc_cds.columns))

except Exception as e:
    print(f"✗ Error loading local file: {e}")
    # --- Fallback: create synthetic 5x5 grid if file not found ---
    from shapely.geometry import Polygon
    import numpy as np

    nyc_bounds = [-74.05, 40.55, -73.70, 40.95]
    grid_cells, names = [], []
    x_min, y_min, x_max, y_max = nyc_bounds
    x_step = (x_max - x_min) / 5
    y_step = (y_max - y_min) / 5

    for i in range(5):
        for j in range(5):
            cell = Polygon([
                (x_min + i * x_step, y_min + j * y_step),
                (x_min + (i+1) * x_step, y_min + j * y_step),
                (x_min + (i+1) * x_step, y_min + (j+1) * y_step),
                (x_min + i * x_step, y_min + (j+1) * y_step)
            ])
            grid_cells.append(cell)
            names.append(f"Grid_{i}_{j}")

    nyc_cds = gpd.GeoDataFrame({'cd_name': names, 'geometry': grid_cells}, crs="EPSG:4326")
    print("✓ Using fallback grid polygons instead of districts")

# Clean up CRS and naming
nyc_cds = nyc_cds.to_crs(gdf.crs)

if 'boro_cd' in nyc_cds.columns and 'boro_name' in nyc_cds.columns:
    nyc_cds['cd_name'] = nyc_cds['boro_name'] + ' CD ' + nyc_cds['boro_cd'].astype(str).str[1:]
elif 'boro_cd' in nyc_cds.columns:
    borough_map = {'1': 'Manhattan', '2': 'Bronx', '3': 'Brooklyn', '4': 'Queens', '5': 'Staten Island'}
    nyc_cds['cd_name'] = nyc_cds['boro_cd'].astype(str).apply(
        lambda x: f"{borough_map.get(x[0], 'Unknown')} CD {x[1:]}" if len(x) == 3 else f"CD_{x}"
    )
else:
    name_col = next((col for col in ['label', 'NAME', 'name', 'CommunityDistrict', 'ntaname', 'boro_name']
                     if col in nyc_cds.columns), None)
    nyc_cds['cd_name'] = nyc_cds[name_col] if name_col else [f'CD_{i}' for i in range(len(nyc_cds))]

print("✓ Community district names assigned")

# Spatial join
print("\nPerforming spatial join (this might take a few minutes)...")
gdf_with_cds = gpd.sjoin(gdf, nyc_cds[['cd_name', 'geometry']], how="left", predicate="within")

# -Calculate pickups per district
pickup_stats = gdf_with_cds.groupby('cd_name').size().reset_index(name='count')
pickup_stats['percentage'] = (pickup_stats['count'] / pickup_stats['count'].sum() * 100).round(2)
top_10 = pickup_stats.nlargest(10, 'count')

# Display results
print("\n" + "="*65)
print("TOP 10 COMMUNITY DISTRICTS BY TAXI PICKUPS")
print("="*65)
for i, (_, row) in enumerate(top_10.iterrows(), 1):
    print(f"{i:2d}. {row['cd_name']:<35} {row['count']:>9,} pickups ({row['percentage']:>5.2f}%)")

print(f"\nTotal pickups analyzed: {pickup_stats['count'].sum():,}")
print(f"Top 10 districts cover {top_10['percentage'].sum():.2f}% of all pickups")

# Borough summary
if 'boro_name' in nyc_cds.columns:
    cd_to_borough = nyc_cds.set_index('cd_name')['boro_name'].to_dict()
    pickup_stats['borough'] = pickup_stats['cd_name'].map(cd_to_borough)
    borough_summary = (
        pickup_stats.groupby('borough')
        .agg({'count': 'sum', 'percentage': 'sum'})
        .sort_values('count', ascending=False)
    )
    print("\n" + "="*50)
    print("PICKUPS BY BOROUGH")
    print("="*50)
    print(borough_summary)


In [ ]:
import folium
import pandas as pd

nyc_cds_map = nyc_cds.merge(pickup_stats, on="cd_name", how="left")

# Drop datetime-like columns (explicitly) and reset index to remove datetime index metadata
for col in nyc_cds_map.columns:
    if pd.api.types.is_datetime64_any_dtype(nyc_cds_map[col]):
        nyc_cds_map[col] = nyc_cds_map[col].astype(str)

nyc_cds_map = nyc_cds_map.reset_index(drop=True)

# Create base map
m = folium.Map(location=[40.75, -73.98], zoom_start=11, tiles="CartoDB positron")

# Choropleth layer
folium.Choropleth(
    geo_data=nyc_cds_map.to_json(),
    data=nyc_cds_map,
    columns=["cd_name", "count"],
    key_on="feature.properties.cd_name",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.4,
    legend_name="Taxi Pickups per Community District"
).add_to(m)

# Add top 10 district markers
for _, row in nyc_cds_map[nyc_cds_map["cd_name"].isin(top_10["cd_name"])].iterrows():
    centroid = row["geometry"].centroid
    popup_text = f"""
    <b>{row['cd_name']}</b><br>
    {int(row['count']):,} pickups<br>
    ({row['percentage']:.1f}% of total)
    """
    folium.CircleMarker(
        location=[centroid.y, centroid.x],
        radius=7,
        color="black",
        fill=True,
        fill_color="red",
        fill_opacity=0.8,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(m)

# Save map to the Figures folder
(d_map_path) = os.path.join(FIG_DIR, "01_nyc_pickups_by_district.html")
m.save(d_map_path)

print(f"💾 Map saved to: {d_map_path}")
m

The choropleth map highlights strong spatial concentration of taxi pickups within Manhattan’s central and western districts, particularly around Midtown, Chelsea, and Lower Manhattan.

These districts display the highest pickup volumes, driven by dense commercial activity, tourism, and transport hubs such as Penn Station and the Financial District.
Demand gradually decreases toward outer boroughs, where residential areas and lower population density reduce ride frequency.

The top 10 community districts account for a majority of all rides, confirming that taxi demand is highly localized and uneven across NYC.
This insight supports the need for district-level forecasting for operational efficiency.

### C. Spatio-Temporal Exploration

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Top 10 districts
top_10 = gdf_with_cds.groupby('cd_name').size().reset_index(name='count').nlargest(10, 'count')
df_top10 = gdf_with_cds[gdf_with_cds['cd_name'].isin(top_10['cd_name'])].copy()

# Hour vs Top 10 districts
hour_district = df_top10.groupby(['Hour','cd_name']).size().reset_index(name='pickups')
hour_pivot = hour_district.pivot(index='Hour', columns='cd_name', values='pickups').fillna(0)

plt.figure(figsize=(12,6))
sns.heatmap(hour_pivot, cmap='Reds', linewidths=.5, annot=True, fmt=".0f")
plt.title('Average Pickups per Hour by Top 10 Districts')
plt.xlabel('District')
plt.ylabel('Hour of Day')
plt.show()

# Weekday vs Top 10 districts
weekday_district = df_top10.groupby(['Weekday','cd_name']).size().reset_index(name='pickups')
weekday_pivot = weekday_district.pivot(index='Weekday', columns='cd_name', values='pickups').fillna(0)

plt.figure(figsize=(12,6))
sns.heatmap(weekday_pivot, cmap='Blues', linewidths=.5, annot=True, fmt=".0f")
plt.title('Average Pickups per Weekday by Top 10 Districts')
plt.xlabel('District')
plt.ylabel('Weekday (0=Mon)')
plt.show()

# Month vs Top 10 districts
month_district = df_top10.groupby(['Month','cd_name']).size().reset_index(name='pickups')
month_pivot = month_district.pivot(index='Month', columns='cd_name', values='pickups').fillna(0)

plt.figure(figsize=(12,6))
sns.heatmap(month_pivot, cmap='Oranges', linewidths=.5, annot=True, fmt=".0f")
plt.title('Average Pickups per Month by Top 10 Districts')
plt.xlabel('District')
plt.ylabel('Month')
plt.show()

Combining hourly, weekly, and monthly trends reveals that NYC taxi demand is both spatially and temporally polarized. Manhattan’s commercial cores exhibit synchronized evening and midweek peaks, while Brooklyn’s districts contribute steady, moderate flows.

Hourly Patterns: Taxi demand peaks sharply during evening hours (17:00–20:00) across all top districts, especially in Midtown and Lower Manhattan (CD 04–05).
Early morning hours (2–5 AM) show minimal activity, reflecting clear commuter and nightlife-driven cycles.

Weekly Patterns: Demand is highest on Tuesdays to Thursdays, tapering slightly over weekends.
Manhattan districts dominate weekday activity, while Brooklyn CDs show steadier weekend demand linked to leisure travel.

Monthly Patterns: Pickups increase from spring to early autumn, peaking around September.
The strongest seasonal growth occurs in central Manhattan, consistent with tourism and business mobility trends.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import os

# Top 10 districts
top_10 = gdf_with_cds.groupby('cd_name').size().reset_index(name='count').nlargest(10, 'count')
df_top10 = gdf_with_cds[gdf_with_cds['cd_name'].isin(top_10['cd_name'])].copy()

# Create EDA tables
hourly_table = (
    df_top10.groupby(['cd_name', 'Hour'])
    .size().reset_index(name='pickups')
    .pivot(index='Hour', columns='cd_name', values='pickups').fillna(0)
)

weekday_table = (
    df_top10.groupby(['cd_name', 'Weekday'])
    .size().reset_index(name='pickups')
    .pivot(index='Weekday', columns='cd_name', values='pickups').fillna(0)
)

day_table = (
    df_top10.groupby(['cd_name', 'Day'])
    .size().reset_index(name='pickups')
    .pivot(index='Day', columns='cd_name', values='pickups').fillna(0)
)

month_table = (
    df_top10.groupby(['cd_name', 'Month'])
    .size().reset_index(name='pickups')
    .pivot(index='Month', columns='cd_name', values='pickups').fillna(0)
)

# Correlation matrix with temporal features
agg = df_top10.groupby(['Hour', 'Weekday', 'Day', 'Month', 'cd_name']).size().reset_index(name='pickups')
pivot = agg.pivot_table(index=['Hour', 'Weekday', 'Day', 'Month'], columns='cd_name', values='pickups', fill_value=0)

pivot['Hour'] = pivot.index.get_level_values('Hour')
pivot['Weekday'] = pivot.index.get_level_values('Weekday')
pivot['Day'] = pivot.index.get_level_values('Day')
pivot['Month'] = pivot.index.get_level_values('Month')

corr_table = pivot.corr()

# Plot and save correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_table, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation of Pickups with Temporal Features")

corr_path = os.path.join(FIG_DIR, "01_correlation_heatmap.png")
plt.savefig(corr_path, dpi=300, bbox_inches="tight")
plt.show()
print(f"💾 Correlation heatmap saved to: {corr_path}")

#  Save to the Figures folder
hourly_table.to_csv(os.path.join(FIG_DIR, "01_hourly_table.csv"))
weekday_table.to_csv(os.path.join(FIG_DIR, "01_weekday_table.csv"))
day_table.to_csv(os.path.join(FIG_DIR, "01_day_table.csv"))
month_table.to_csv(os.path.join(FIG_DIR, "01_month_table.csv"))
corr_table.to_csv(os.path.join(FIG_DIR, "01_correlation_table.csv"))

print("✅ All tables and figures saved in:", FIG_DIR)


The heatmap shows strong positive correlations among most Manhattan districts (r ≈ 0.85–0.9) — indicating that demand patterns across central Manhattan move together.

Brooklyn districts display moderate correlations (r ≈ 0.6–0.75) with each other and weaker links to Manhattan zones, reflecting distinct local demand dynamics.

Temporal features like Hour show moderate correlations (r ≈ 0.3–0.6) with Manhattan districts, which makes time-based cycles worth exploring in urban areas, while Weekday and Day contribute little direct variance.

Overall, this suggests that spatial proximity can drive synchronized demand, while temporal variation mainly amplifies existing Manhattan-centric trends.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf, pacf

# Aggregate hourly data
time_series = (
    df_top10.groupby(['cd_name', 'Hour', 'Weekday', 'Day', 'Month'])
    .size()
    .reset_index(name='pickups')
)

# Create a pseudo time index
time_series['timestamp'] = (
    time_series['Month'].astype(str).str.zfill(2) + '-' +
    time_series['Day'].astype(str).str.zfill(2) + ' ' +
    time_series['Hour'].astype(str).str.zfill(2)
)
time_series['timestamp'] = pd.to_datetime(time_series['timestamp'], format='%m-%d %H', errors='coerce')

#  ACF & PACF calculation
acf_results = {}
pacf_results = {}
max_lag = 48  # 2 days

for district in top_10['cd_name']:
    district_data = (
        time_series[time_series['cd_name'] == district]
        .sort_values('timestamp')
        ['pickups']
        .fillna(0)
        .to_numpy()
    )

    if len(district_data) < max_lag:
        print(f"⚠️ Skipping {district}: insufficient data ({len(district_data)} points)")
        continue

    acf_vals = acf(district_data, nlags=max_lag, fft=True)
    pacf_vals = pacf(district_data, nlags=max_lag, method='yw')

    acf_results[district] = acf_vals
    pacf_results[district] = pacf_vals

#  Convert to DataFrames
lags = np.arange(0, max_lag + 1)
acf_df = pd.DataFrame(acf_results, index=lags).reset_index().rename(columns={'index': 'Lag'})
pacf_df = pd.DataFrame(pacf_results, index=lags).reset_index().rename(columns={'index': 'Lag'})

# Export CSV
acf_csv = os.path.join(FIG_DIR, "01_acf_results_top10.csv")
pacf_csv = os.path.join(FIG_DIR, "01_pacf_results_top10.csv")

print("✅ Exported CSVs:")
print("01_acf_results_top10.csv")
print("01_pacf_results_top10.csv")

# Visualization
num_districts = len(acf_results)
fig, axes = plt.subplots(num_districts, 2, figsize=(12, 4 * num_districts))

for i, district in enumerate(acf_results.keys()):
    # Confidence bounds
    N = len(time_series[time_series['cd_name'] == district])
    conf = 1.96 / np.sqrt(N)

    # ACF
    axes[i, 0].stem(lags, acf_results[district], linefmt='b-', markerfmt='bo', basefmt='r-')
    axes[i, 0].axhline(y=conf, color='r', linestyle='--', linewidth=0.8)
    axes[i, 0].axhline(y=-conf, color='r', linestyle='--', linewidth=0.8)
    axes[i, 0].set_title(f"{district} – ACF", fontsize=11)
    axes[i, 0].set_xlabel("Lag (hours)")
    axes[i, 0].set_ylabel("Correlation")

    # PACF
    axes[i, 1].stem(lags, pacf_results[district], linefmt='g-', markerfmt='go', basefmt='r-')
    axes[i, 1].axhline(y=conf, color='r', linestyle='--', linewidth=0.8)
    axes[i, 1].axhline(y=-conf, color='r', linestyle='--', linewidth=0.8)
    axes[i, 1].set_title(f"{district} – PACF", fontsize=11)
    axes[i, 1].set_xlabel("Lag (hours)")
    axes[i, 1].set_ylabel("Partial Correlation")

plt.tight_layout()
plt.show()

# Save to the Figures folder
acf_pacf_path = os.path.join(FIG_DIR, "01_acf_pacf_top10.png")
plt.savefig(acf_pacf_path, dpi=300, bbox_inches="tight")
plt.show()
print(f"💾 ACF/PACF plots saved to: {acf_pacf_path}")

The ACF plots show strong short-lag autocorrelation across nearly all districts, confirming that current taxi demand is highly dependent on recent hours.
Distinct spikes at lag = 24 indicate clear daily seasonality, reflecting the repeated 24-hour pickup cycle.

PACF plots reinforce this pattern, with initial lags remaining significant before tapering off, suggesting that only a few recent time steps (typically within 1–3 hours) contribute most to predictive power.

These findings validate the use of lag features and sequence-based models (e.g., LSTM, Conv1D) for hourly demand forecasting.